# 3D人体关键点 预训练模型预测

参考文档：https://github.com/open-mmlab/mmpose/blob/master/demo/docs/3d_human_pose_demo.md

作者：同济子豪兄 2022-06-06

如果报错`CUDA out of memory.`则重启前面几个代码的`kernel`即可。

## 进入 MMPose 主目录

In [1]:
import os
os.chdir('mmpose')

## 3D Human Pose的`two-stage`算法

第一阶段：2D人体关键点检测

第二阶段：2D转3D映射

In [2]:
!python demo/body3d_two_stage_img_demo.py \
        configs/body/3d_kpt_sview_rgb_img/pose_lift/h36m/simplebaseline3d_h36m.py \
        https://download.openmmlab.com/mmpose/body3d/simple_baseline/simple3Dbaseline_h36m-f0ad73a4_20210419.pth \
        --json-file tests/data/h36m/h36m_coco.json \
        --img-root tests/data/h36m \
        --camera-param-file tests/data/h36m/cameras.pkl \
        --only-second-stage \
        --out-img-root outputs/B3/B3_1_3d_img \
        --rebase-keypoint-height \
        --show-ground-truth

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Stage 1: load 2D pose results from Json file.
Stage 2: 2D-to-3D pose lifting.
load checkpoint from http path: https://download.openmmlab.com/mmpose/body3d/simple_baseline/simple3Dbaseline_h36m-f0ad73a4_20210419.pth
Downloading: "https://download.openmmlab.com/mmpose/body3d/simple_baseline/simple3Dbaseline_h36m-f0ad73a4_20210419.pth" to /home/featurize/.cache/torch/hub/checkpoints/simple3Dbaseline_h36m-f0ad73a4_20210419.pth
100%|██████████████████████████████████████| 16.4M/16.4M [00:00<00:00, 74.2MB/s]
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 4/4, 5.4 task/s, elapsed: 1s, ETA:     0s


## 视频预测

命令行中的三个模型分别是：

2D 人体框检测

2D 人体关键点检测

2d-to-3d pose lifting 直接用2D坐标回归3D坐标，把2D的关键点坐标映射到三维

### 单帧输入模型的视频预测

In [2]:
!python demo/body3d_two_stage_video_demo.py \
        demo/mmdetection_cfg/faster_rcnn_r50_fpn_coco.py \
        https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth \
        configs/body/2d_kpt_sview_rgb_img/topdown_heatmap/coco/hrnet_w48_coco_256x192.py \
        https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w48_coco_256x192-b9e0b3ab_20200708.pth \
        configs/body/3d_kpt_sview_rgb_vid/video_pose_lift/h36m/videopose3d_h36m_243frames_fullconv_supervised_cpn_ft.py \
        https://download.openmmlab.com/mmpose/body3d/videopose/videopose_h36m_243frames_fullconv_supervised_cpn_ft-88f5abbb_20210527.pth \
        --video-path data/two_dancers.mp4 \
        --out-video-root outputs/B3/B3_2_3d_video \
        --rebase-keypoint-height

Stage 1: 2D pose detection.
Initializing model...
load checkpoint from http path: https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth
load checkpoint from http path: https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w48_coco_256x192-b9e0b3ab_20200708.pth
Running 2D pose detection inference...
[>>>>>>>>>>>>>>>>>>>>>>>>>>>> ] 554/563, 5.5 task/s, elapsed: 101s, ETA:     2s
Stage 2: 2D-to-3D pose lifting.
Initializing model...
load checkpoint from http path: https://download.openmmlab.com/mmpose/body3d/videopose/videopose_h36m_243frames_fullconv_supervised_cpn_ft-88f5abbb_20210527.pth
Downloading: "https://download.openmmlab.com/mmpose/body3d/videopose/videopose_h36m_243frames_fullconv_supervised_cpn_ft-88f5abbb_20210527.pth" to /home/featurize/.cache/torch/hub/checkpoints/videopose_h36m_243frames_fullconv_supervised_cpn_ft-88f5abbb_20210527.pth
100%|███████████████████████████████████████| 64.8

### 多帧输入模型的视频预测

In [6]:
!python demo/body3d_two_stage_video_demo.py \
    demo/mmdetection_cfg/faster_rcnn_r50_fpn_coco.py \
    https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth \
    configs/body/2d_kpt_sview_rgb_vid/posewarper/posetrack18/hrnet_w48_posetrack18_384x288_posewarper_stage2.py \
    https://download.openmmlab.com/mmpose/top_down/posewarper/hrnet_w48_posetrack18_384x288_posewarper_stage2-4abf88db_20211130.pth  \
    configs/body/3d_kpt_sview_rgb_vid/video_pose_lift/h36m/videopose3d_h36m_243frames_fullconv_supervised_cpn_ft.py \
    https://download.openmmlab.com/mmpose/body3d/videopose/videopose_h36m_243frames_fullconv_supervised_cpn_ft-88f5abbb_20210527.pth \
    --video-path data/two_dancers.mp4 \
    --out-video-root outputs/B3/B3_3_3d_video_multi_frames \
    --rebase-keypoint-height \
    --use-multi-frames --online

Stage 1: 2D pose detection.
Initializing model...
load checkpoint from http path: https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth
load checkpoint from http path: https://download.openmmlab.com/mmpose/top_down/posewarper/hrnet_w48_posetrack18_384x288_posewarper_stage2-4abf88db_20211130.pth
Downloading: "https://download.openmmlab.com/mmpose/top_down/posewarper/hrnet_w48_posetrack18_384x288_posewarper_stage2-4abf88db_20211130.pth" to /home/featurize/.cache/torch/hub/checkpoints/hrnet_w48_posetrack18_384x288_posewarper_stage2-4abf88db_20211130.pth
100%|█████████████████████████████████████████| 273M/273M [00:02<00:00, 107MB/s]
Running 2D pose detection inference...
[>>>>>>>>>>>>>>>>>>>>>>>>>>>> ] 554/563, 2.3 task/s, elapsed: 237s, ETA:     4s
Stage 2: 2D-to-3D pose lifting.
Initializing model...
load checkpoint from http path: https://download.openmmlab.com/mmpose/body3d/videopose/videopose_h36m_243